In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
# from nltk.lm import Vocabulary
# vocabulary = Vocabulary(text)
# len(vocabulary)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 17
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])

556949
556932
['\ufeffThe Project Gute', 'The Project Guten', 'he Project Gutenb', 'e Project Gutenbe', ' Project Gutenber']


# fit

In [8]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(vocabulary)
net.to(device)

encoded = net.text2tensor(lines).to(device)
print(encoded.shape)

performance = net.fit(encoded, save_path = "checkpoints/autoregressive_lstm.pt", progress_bar = 0)
net.save_architecture("models/autoregressive_lstm.arch")

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

torch.Size([556932, 17])
Training started
Epochs: 5
Learning rate: 0.001
Weight decay: 0
Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------
    1 |   1.2248 |     34.568 |     0.7
    2 |   0.9714 |     29.082 |     1.5
    3 |   0.9251 |     27.978 |     2.3
    4 |   0.8987 |     27.328 |     3.1
    5 |   0.8803 |     26.873 |     3.9


In [9]:
# includes all the information about the epoch and the model, useful for reproducibility

performance

,epoch,train_loss,train_error_rate,training_minutes,learning_rate,weight_decay,model,embedding_dimension,hidden_units,layers,dropout,parameters
0,1,1.224767,34.567708,0.748852,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
1,2,0.971367,29.081737,1.526219,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
2,3,0.925073,27.978281,2.304914,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
3,4,0.898749,27.327753,3.073714,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
4,5,0.880287,26.872659,3.851194,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335


In [10]:
# the input for testing

net.tensor2text(encoded[:5])

['\ufeffThe Project Gute',
 'The Project Guten',
 'he Project Gutenb',
 'e Project Gutenbe',
 ' Project Gutenber']

# predict

In [11]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("models/autoregressive_lstm.arch")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
idx, log_probabilities = net.predict(encoded[:5], main_progress_bar = False, progress_bar = 0)

net.tensor2text(idx)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335



['\ufeffThe Project Gutenberg Literary Archi',
 'The Project Gutenberg Literary Archiv',
 'he Project Gutenberg Literary Archive',
 'e Project Gutenberg Literary Archive ',
 ' Project Gutenberg Literary Archive F']

In [12]:
log_probabilities

tensor([[-2.2667, -2.6597, -4.9379, -5.1540, -5.4737],
        [-2.3132, -2.7272, -5.0683, -5.3617, -6.3552],
        [-2.3244, -2.7650, -5.1401, -5.6308, -6.4696],
        [-2.4750, -2.7939, -4.4382, -5.2172, -5.6364],
        [-2.7056, -2.8104, -5.1420, -5.2348, -5.4865]], device='cuda:0')

# greedy_search

In [13]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("models/autoregressive_lstm.arch")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([-2.6597, -2.7272, -2.7650, -2.7939, -2.8104], device='cuda:0')


['\ufeffThe Project Gutenberg-tm electronic ',
 'The Project Gutenberg-tm electronic w',
 'he Project Gutenberg-tm electronic wo',
 'e Project Gutenberg-tm electronic wor',
 ' Project Gutenberg-tm electronic work']

# sample

In [14]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("models/autoregressive_lstm.arch")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([-13.3856, -12.2580,  -9.9078, -22.7814, -22.9671], device='cuda:0')


['\ufeffThe Project Gutenberg-tm work, sever',
 'The Project Gutenberg-tm\ncasker.\n\n   ',
 'he Project Gutenberg-tm License is to',
 'e Project Gutenberg Literably the uls',
 ' Project Gutenberg White gives P-KB4 ']

# beam_search

In [15]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("models/autoregressive_lstm.arch")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5], progress_bar = False)

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([[-2.2667, -2.6597, -4.9379, -5.1540, -5.4737],
        [-2.3132, -2.7272, -5.0683, -5.3617, -6.3552],
        [-2.3244, -2.7650, -5.1401, -5.6308, -6.4696],
        [-2.4750, -2.7939, -4.4382, -5.2172, -5.6364],
        [-2.7056, -2.8104, -5.1420, -5.2348, -5.4865]], device='cuda:0')
[['\ufeffThe Project Gutenberg Literary Archi',
  '\ufeffThe Project Gutenberg-tm electronic ',
  '\ufeffThe Project Gutenberg-tm electronic\n',
  '\ufeffThe Project Gutenberg-tm electronica',
  '\ufeffThe Project Gutenberg-tm trademark, '],
 ['The Project Gutenberg Literary Archiv',
  'The Project Gutenberg-tm electronic w',
  'The Project Gutenberg-tm electronic\nw',
  'The Project Gutenberg-tm electronical',
  'The Project Gutenberg Literary Archim'],
 ['he Project Gutenberg Literary Archive',
  'he Project Gutenberg-tm electronic wo',
  'he Proj